<a href="https://colab.research.google.com/github/ayaanzhaque/SuiSense/blob/master/notebooks/BERT/workingBertModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers

     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 9.7MB/s 
     |████████████████████████████████| 890kB 16.4MB/s 
     |████████████████████████████████| 3.0MB 23.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a42506fcf8ff7e95c5f0f8aaedbaaa7a95ffb29208451a65f9b4c3c3ad6b46e6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
#importing relevant libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

import torch

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset
We'll use pandas to read the dataset and load it into a dataframe.

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/ayaanzhaque/SuiSense/master/data/scheme1.csv')
og_batch_1 = df[['selftext', 'New Labels']]



For performance reasons, we'll only use 2,000 sentences from the dataset

In [6]:
#batch_1_start = og_batch_1.head(60)
#batch_1_end = og_batch_1.tail(60)
#test_batch_1 = pd.concat([batch_1_start, batch_1_end], ignore_index=True)
batch_1 = og_batch_1.rename(columns={'selftext': 0, 'New Labels': 1})
batch_1.head()

,0,1
0,We understand that most people who reply immed...,1
1,Welcome to /r/depression's check-in post - a p...,1
2,I've been feeling really depressed and lonely ...,0
3,I literally broke down crying and asked to go ...,0
4,Any kind soul want to give a depressed person ...,1


We can ask pandas how many sentences are labeled as "positive" (value 1) and how many are labeled "negative" (having the value 0)

In [7]:
batch_1[1].value_counts()

1    1446
0     451
Name: 1, dtype: int64

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [8]:
# For DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [9]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=128)))

In [10]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [11]:
np.array(padded).shape

(1897, 128)

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [12]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1897, 128)

In [14]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [15]:
features = last_hidden_states[0][:,0,:].numpy()
print(features)

[[ 0.15627055 -0.20722187 -0.03962998 ...  0.15881488  0.3669506
   0.46319035]
 [-0.05789474 -0.01733885 -0.1070803  ...  0.01880805 -0.06970198
   0.54524016]
 [ 0.19604996  0.20534162  0.24803358 ...  0.1578196   0.45455894
  -0.4178381 ]
 ...
 [ 0.04628709  0.53464365 -0.19218428 ... -0.3401183   0.7708875
   0.6021303 ]
 [ 0.247009    0.28108364  0.2101751  ... -0.19128986  0.26677406
  -0.13201515]
 [ 0.16641271  0.16862686  0.03572455 ... -0.15101437  0.5139445
   0.24030793]]


The labels indicating which sentence is positive and negative now go into the `labels` variable

In [16]:
labels = batch_1[1]
labels.head()

0    1
1    1
2    0
3    0
4    1
Name: 1, dtype: int64

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 2,000 sentences from the SST2 training set).

In [17]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.25, random_state=42, stratify=labels)

In [18]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. `LogisticRegression(C=5.2)`).

In [28]:
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ModelCheckpoint

model3 = Sequential()
model3.add(Dense(1, activation='sigmoid'))

model3.add(Dense(10, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))

model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


checkpoint = ModelCheckpoint('/content/bertmodel.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  


history = model3.fit(train_features, train_labels, batch_size=32, epochs=100, verbose=1, callbacks=[checkpoint], validation_data=(test_features, test_labels))

Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 0.5892 - accuracy: 0.7623
Epoch 00001: val_loss improved from inf to 0.56322, saving model to /content/bertmodel.h5
45/45 [==============================] - 0s 4ms/step - loss: 0.5892 - accuracy: 0.7623 - val_loss: 0.5632 - val_accuracy: 0.7621
Epoch 2/100
42/45 [===========================>..] - ETA: 0s - loss: 0.5586 - accuracy: 0.7604
Epoch 00002: val_loss improved from 0.56322 to 0.55202, saving model to /content/bertmodel.h5
45/45 [==============================] - 0s 2ms/step - loss: 0.5565 - accuracy: 0.7623 - val_loss: 0.5520 - val_accuracy: 0.7621
Epoch 3/100
42/45 [===========================>..] - ETA: 0s - loss: 0.5458 - accuracy: 0.7664
Epoch 00003: val_loss improved from 0.55202 to 0.54779, saving model to /content/bertmodel.h5
45/45 [==============================] - 0s 2ms/step - loss: 0.5500 - accuracy: 0.7623 - val_loss: 0.5478 - val_accuracy: 0.7621
Epoch 4/100
39/45 [=========================>....]

<img src="https://jalammar.github.io/images/distilBERT/bert-training-logistic-regression.png" />

## Evaluating Model #2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [27]:
from keras.models import load_model

loadedModel = load_model('/content/bertmodel.h5')
loadedModel.score(test_features, test_labels)

AttributeError: ignored

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

So our model clearly does better than a dummy classifier. But how does it compare against the best models?

## Proper SST2 scores
For reference, the [highest accuracy score](http://nlpprogress.com/english/sentiment_analysis.html) for this dataset is currently **96.8**. DistilBERT can be trained to improve its score on this task – a process called **fine-tuning** which updates BERT’s weights to make it achieve a better performance in this sentence classification task (which we can call the downstream task). The fine-tuned DistilBERT turns out to achieve an accuracy score of **90.7**. The full size BERT model achieves **94.9**.



And that’s it! That’s a good first contact with BERT. The next step would be to head over to the documentation and try your hand at [fine-tuning](https://huggingface.co/transformers/examples.html#glue). You can also go back and switch from distilBERT to BERT and see how that works.